In [1]:
import gzip
import mlflow
import pandas as pd
import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from gensim import corpora
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm, trange

use_cuda = torch.cuda.is_available()
device = torch.device('cuda') if use_cuda else torch.device('cpu')

# Dataset

In [2]:
class MeLiDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.to_list()
        
        item = {
            "data": self.dataset.loc[item, "title"],
            "target": self.dataset.loc[item, "category"]
        }
        
        if self.transform:
            item = self.transform(item)
        
        return item

# Preprocesamiento

In [3]:
class RawDataProcessor:
    def __init__(self, 
                 dataset, 
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=500000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
#                 preprocessing.strip_tags,
#                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
#                 preprocessing.strip_numeric,
#                 preprocessing.remove_stopwords,
#                 preprocessing.strip_short,
            ]
        
        # Create dictionary based on all the reviews (with corresponding preprocessing)
        self.dictionary = corpora.Dictionary(
            dataset["title"].map(self._preprocess_string).tolist()
        )
        # Filter the dictionary and compactify it (make the indices continous)
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        self.dictionary.compactify()
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        })
        self.idx_to_target = sorted(dataset["category"].unique())
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}

    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx[target]
    
    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self.encode_data(item["data"])
        else:
            data = [self.encode_data(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self.encode_target(item["target"])
        else:
            target = [self.encode_target(t) for t in item["target"]]
        
        return {
            "x": data,
            "y": target,
            "l": len(data)
        }

# Lectura de datos

In [4]:
train_dataset = pd.read_csv("./spanish.train.csv.gz")
test_dataset = pd.read_csv("./spanish.test.csv.gz")

In [5]:
preprocess = RawDataProcessor(train_dataset)

# train_indices, test_indices = train_test_split(dataset.index, test_size=0.2, random_state=42)

train_dataset = MeLiDataset(train_dataset[0:10], transform=preprocess)

test_dataset = MeLiDataset(test_dataset, transform=preprocess)

print(f"Datasets loaded with {len(train_dataset)} training elements and {len(test_dataset)} test elements")
print(f"Sample train element:\n{train_dataset[0]}")

Datasets loaded with 10 training elements and 63680 test elements
Sample train element:
{'x': [500001, 2, 500000, 3, 4], 'y': 196, 'l': 5}


In [6]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        if items is None:
            return (None,None,None)
        data, target, length = list(zip(*[(item["x"], item["y"], item["l"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return torch.LongTensor(data), torch.FloatTensor(target), torch.IntTensor(length)
        

# DataLoaders

In [7]:
pad_sequences = PadSequences()
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True,
                          collate_fn=pad_sequences, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False,
                         collate_fn=pad_sequences, drop_last=False)

# Training Loop

# RNN Model

In [8]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class MeLiRNNClassifier(torch.nn.Module) :
    def __init__(self, pretrained_embeddings_path, 
                 dictionary,
                 vector_size,
                 freeze_embedings,
                 hidden_dim,
                dropout=0.3, output_layer=632):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout)
        
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with gzip.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        
        self.lstm = nn.LSTM(vector_size, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_layer)
        
    def forward(self, x, s):
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = torch.sigmoid((self.linear(ht[-1])))
        return out


In [9]:
import numpy as np
from sklearn.metrics import mean_squared_error

def train_model(model, epochs=1, lr=0.001):
    mlflow.set_experiment("AprendizajeProfundo_RNN_experiment")
    mlflow.log_param("model_name", "MeLiRNNClassifier")
    mlflow.log_params({
        "embedding_size": 50,
        "hidden_dim": 50,
        "dropout": 0.3
    })
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        running_loss = []
        for idx, batch in enumerate(tqdm(train_loader)):
            x = batch[0].long().to(device)
            y = batch[1].long().to(device)
            y_pred = model(x, batch[2])
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
            running_loss.append(loss.item())
        
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epochs)
        
#         model.eval()
#         correct = 0
#         total = 0
#         sum_loss = 0.0
#         sum_rmse = 0.0
#         running_loss = []
#         for idx, batch in enumerate(tqdm(test_loader)):
#             x = batch[0].long().to(device)
#             y = batch[1].long().to(device)
#             y_hat = model(x, batch[2])
#             loss = F.cross_entropy(y_hat, y)
#             pred = torch.max(y_hat, 1)[1]
#             correct += (pred == y).float().sum()
#             total += y.shape[0]
#             sum_loss += loss.item()*y.shape[0]
#             sum_rmse += np.sqrt(mean_squared_error(pred.cpu(), y.cpu().unsqueeze(-1)))*y.cpu().shape[0]
#             running_loss.append(loss.item())

#         mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epochs)
#         mlflow.log_metric("test_bacc", balanced_accuracy_score(y.cpu(), y_hat.cpu(), epochs))

        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            output = model(batch[0].long().to(device), batch[2])
            running_loss.append(
                F.cross_entropy(output, batch[1].long().to(device))
            )
            targets.extend(batch[1].cpu().numpy())
            predictions.extend(output.cpu().squeeze().detach().numpy())
        mlflow.log_metric("test_loss", float(sum(running_loss) / len(running_loss)), epochs)
        mlflow.log_metric("test_bacc", balanced_accuracy_score(targets, predictions), epochs)

    with tempfile.TemporaryDirectory() as tmpdirname:
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            output = model(batch["x"].cpu(), batch["l"].cpu())
            targets.extend(batch["y"].cpu().numpy())
            predictions.extend(output.squeeze().detach().numpy())
        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
            f"{tmpdirname}/predictions.csv.gz", index=False
        )
        mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

In [10]:
embedding_size = 50
freeze_embedding = True

model = MeLiRNNClassifier("./glove.6B.50d.txt.gz", preprocess.dictionary, embedding_size, freeze_embedding, 1).to(device)

train_model(model, epochs=5, lr=0.05)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets